In [1]:
import os
from io import BytesIO
import os.path
import dask.dataframe as dd
import pandas as pd
import praw
import requests
from PIL import Image
from tqdm import tqdm
import shutil
import os.path
import pandas
from bigtree import Node, print_tree
from tqdm import notebook
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.storage import azure_file_storage

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter

tqdm.pandas()
tqdm.pandas(desc="global")

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [ ]:
from typing import Union

def update_data_frame_by_id(df: pd.DataFrame, id_value: str, column: str, value: Union[str,bool,int,None,list]) -> pd.DataFrame:
	df.loc[df['id'] == id_value, column] = value
	return df

In [ ]:
def filter_on_existing_thumbnail_path(x: object) -> bool:
	try:
		path = x['thumbnail_path']
		return os.path.exists(path)
	except:
		return False

In [ ]:
def get_image_bytes(x: object) -> object:
	try:
		path = x['path']
		data = open(path, 'rb').read()
		return data
	except:
		return None

In [1]:
sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": [ "selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor" ]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion", "data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion", "data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]}
]
sources_df = pd.DataFrame.from_records(sources)

NameError: name 'pd' is not defined

In [ ]:
def make_alt_thumbnail(subreddit, md5_sum, image_path) -> str:
	subreddit = subreddit
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5_sum}.jpg"
	max_size = (512, 512)

	if image_path is None:
		return ""
	if md5_sum is None:
		return ""
	if subreddit is None:
		return ""

	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path

	try:
		img = Image.open(image_path)
		copy = img.copy()
		copy.thumbnail(max_size, Image.LANCZOS)
		copy.save(out_path)
		copy.close()
		img.close()
		return out_path

	except Exception as e:
		print(e)
		return ""

In [ ]:
def check_if_exists(thumbnail_path):
	if thumbnail_path is None:
		return False
	if not os.path.exists(thumbnail_path):
		return False
	return True

In [ ]:
def check_if_thumb_and_image_exist(x: object):
	try:
		path = x['path']
		if path is None:
			return False
		thumb_nail_path = x['thumbnail_path']
		if thumb_nail_path is None:
			return False
		full_image_exists = os.path.exists(path)
		thumbnail_image_exists = os.path.exists(thumb_nail_path)
		return full_image_exists and thumbnail_image_exists
	except Exception as e:
		print("Error In check_if_thumb_and_image_exist: " + str(e))
		return False

In [ ]:
def move_file(x: object) -> str:
	try:
		if not os.path.exists(x['path']):
			return ""
		else:
			out_path = f"{x['name']}/{x['image_name']}"
			shutil.copy2(x['path'], out_path)
			return out_path
	except Exception as e:
		print(e)
		return ""

In [ ]:
import hashlib
def fetch_image(subreddit: str, url: str, known_hash_list: [str]) -> object:
	try:
		response = requests.get(url)
		md5_ = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5_}.jpg"

		if os.path.exists(out_path):
			small_out_path = make_thumbnail(subreddit, md5_, out_path)
			return out_path, small_out_path, md5_

		if md5_ in known_hash_list:
			small_out_path = make_thumbnail(subreddit, md5_, out_path)
			return out_path, small_out_path, md5_

		if md5_ != "f17b01901c752c1bb04928131d1661af" or md5_ != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.convert('RGB')
			raw_image.save(out_path)
			raw_image.close()
			small_out_path = make_thumbnail(subreddit, md5_, out_path)
			return out_path, small_out_path, md5_
		else:
			return "", "", ""

	except Exception as e:
		print(e)
		return "", "", ""

In [ ]:
def make_thumbnail(subreddit, md5_thing, original_image_path) -> str:
	original_image_path = original_image_path
	md5_ = md5_thing
	subreddit = subreddit
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5_}.jpg"
	if not os.path.exists(original_image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(original_image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		print(e)
		return ""
	finally:
		img.close()

In [ ]:
def add_source(x: object, source_list) -> str:
	for source in source_list:
		if x['subreddit'] in source['data']:
			return source['name']
	return ""

In [ ]:
from dataclasses import dataclass

from typing import List


@dataclass
class DataSources:
	name: str
	data: List[str]

	@staticmethod
	def from_dict(obj: dict) -> 'DataSources':
		_name = obj.get("name")
		_data = [x for x in obj.get("data")]
		return DataSources(_name, _data)